## 使用kaldi中x-vector方法在aishell数据库上训练说话人识别模型
    
    VAD，MFCC特征
    采用数据增强手段 ，加入噪音，模拟混响
    x-vector
    LDA,PLDA
    EER,minDCF
   

**第一步：** 准备数据配置索引文件

    使用脚本生成下面的索引文件
    
    data
    ├── test
    │   ├── spk2utt
    │   ├── text
    │   ├── utt2spk
    │   └── wav.scp
    └── train
        ├── spk2utt
        ├── text
        ├── utt2spk
        └── wav.scp

In [1]:
!chmod +x new_01_prepare_train_index.sh
!./new_01_prepare_train_index.sh

Preparing data/local/train transcriptions
Preparing data/local/test transcriptions
local/aishell_data_prep.sh: AISHELL data preparation succeeded


**第二步：** 准备原始语音的mfcc表征，并进行VAD操作

    mfcc的参数：
    --sample-frequency=16000 
    --frame-length=25 # the default is 25
    --low-freq=20 # the default.
    --high-freq=8000 # the default is zero meaning use the Nyquist (4k in this case).
    --num-ceps=23 # higher than the default which is 12.
    --snip-edges=false
    vad的参数：
    --vad-energy-threshold=5.5
    --vad-energy-mean-scale=0.5
    --vad-proportion-threshold=0.12
    --vad-frames-context=2


In [3]:
import time

ss = time.time()
!chmod +x new_02_generate_mfcc_and_vad.sh
!./new_02_generate_mfcc_and_vad.sh

print("耗时：",time.time() - ss)

## 用时：1150s，

steps/make_mfcc.sh --write-utt2num-frames true --mfcc-config conf/mfcc.conf --nj 20 --cmd run.pl --mem 10G data/train exp/make_mfcc feature/mfcc
utils/validate_data_dir.sh: Successfully validated data-directory data/train
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for train
fix_data_dir.sh: kept all 120098 utterances.
fix_data_dir.sh: old files are kept in data/train/.backup
sid/compute_vad_decision.sh --nj 20 --cmd run.pl --mem 10G data/train exp/make_vad feature/mfcc
Created VAD output for train
fix_data_dir.sh: kept all 120098 utterances.
fix_data_dir.sh: old files are kept in data/train/.backup
steps/make_mfcc.sh --write-utt2num-frames true --mfcc-config conf/mfcc.conf --nj 20 --cmd run.pl --mem 10G data/test exp/make_mfcc feature/mfcc
utils/validate_data_dir.sh: Successfully validated data-directory data/test
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp in

**第三步：** 生成带有混响和噪音的索引文件

    1. 按照1：1的数量，生成带有混响的语音
    2. 按照1：3的数量，生成带有各种噪音的语音
    
    data中出现增加混响的配置文件，增加噪音的配置文件（data/train_reverb data/train_noise data/train_music data/train_babble）
    将4种增强的数据组合到一个文件中data/train_aug
    注意，此时并没有提取mfcc特征，仅仅是设计了如何扩充数据。

In [8]:
ss = time.time()

!chmod +x new_03_prepare_aug_data.sh
!./new_03_prepare_aug_data.sh

print("耗时：",time.time() - ss)

steps/data/reverberate_data_dir.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import argparse, shlex, glob, math, os, random, sys, warnings, copy, imp, ast
steps/data/reverberate_data_dir.py --rir-set-parameters 0.5, /home/qinyc/openSLR/SLR28_rir/RIRS_NOISES/simulated_rirs/smallroom/rir_list --rir-set-parameters 0.5, /home/qinyc/openSLR/SLR28_rir/RIRS_NOISES/simulated_rirs/mediumroom/rir_list --speech-rvb-probability 1 --pointsource-noise-addition-probability 0 --isotropic-noise-addition-probability 0 --num-replications 1 --source-sampling-rate 16000 data/train data/train_reverb
Number of RIRs is 40000
Getting the duration of the recordings...
utils/copy_data_dir.sh: copied data from data/train_reverb to data/train_reverb.new
utils/validate_data_dir.sh: Successfully validated data-directory data/train_reverb.new
./new_03_prepare_dataaugment.sh: 19: ./new_03_prepare_dataaugment.sh: Prepare: not found


**第四步：** 将第三步中扩充的数据提取为MFCC表征

    数据同样保存在mfcc这个文件中，日志文件存放在exp/make_mfcc
   

In [9]:
ss = time.time()

!chmod +x new_04_extract_feature.sh
!./new_04_extract_feature.sh

print("耗时：",time.time() - ss)

steps/make_mfcc.sh --mfcc-config conf/mfcc.conf --nj 20 --cmd run.pl --mem 10G data/train_aug exp/make_mfcc mfcc
utils/validate_data_dir.sh: Successfully validated data-directory data/train_aug
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for train_aug
耗时： 8234.177950143814


**第五步：** 过滤语音
    
    ## 1. 将train_aug数据和train数据合并
    ## 2. 该脚本应用CMVN并删除非语音帧。 会重新复制一份feature，确保磁盘空间。
    ## 3. 删除静音帧后删除过短的功能。200帧
    ## 4. 去除语音数量少于8个的说话人。
        
        
    

In [10]:
ss = time.time()

!chmod +x new_05_filter_feature.sh
!./new_05_filter_feature.sh

print("耗时：",time.time() - ss)

utils/combine_data.sh data/train_combined data/train_aug data/train
utils/combine_data.sh: combined utt2uniq
utils/combine_data.sh [info]: not combining segments as it does not exist
utils/combine_data.sh: combined utt2spk
utils/combine_data.sh [info]: not combining utt2lang as it does not exist
utils/combine_data.sh: combined utt2dur
utils/combine_data.sh: combined utt2num_frames
utils/combine_data.sh [info]: **not combining reco2dur as it does not exist everywhere**
utils/combine_data.sh: combined feats.scp
utils/combine_data.sh: combined text
utils/combine_data.sh [info]: not combining cmvn.scp as it does not exist
utils/combine_data.sh: combined vad.scp
utils/combine_data.sh [info]: not combining reco2file_and_channel as it does not exist
utils/combine_data.sh: combined wav.scp
utils/combine_data.sh [info]: not combining spk2gender as it does not exist
fix_data_dir.sh: kept all 600490 utterances.
fix_data_dir.sh: old files are kept in data/train_combined/.backup
fix_data_dir.sh: ke

**第六步：** 建立训练模型
    
    1.构建模型的结构
    2.训练模型
    
    生成了最终的x-vector 模型，final.raw

In [13]:
!/home/amax/Codes/softwares/kaldi-trunk/src/nnet3bin/nnet3-init ./exp/xvector_nnet_1a/nnet.config ./exp/xvector_nnet_1a/0.raw

/home/amax/Codes/softwares/kaldi-trunk/src/nnet3bin/nnet3-init ./exp/xvector_nnet_1a/nnet.config ./exp/xvector_nnet_1a/0.raw 
LOG (nnet3-init[5.5.480~1-09abd]:main():nnet3-init.cc:80) Initialized raw neural net and wrote it to ./exp/xvector_nnet_1a/0.raw


In [15]:
ss = time.time()

!chmod +x new_06_train_x-vector.sh
!bash new_06_train_x-vector.sh --stage 0 --train-stage 0 --data data/train_combined_no_sil --nnet-dir exp/xvector_nnet_1a --egs-dir exp/xvector_nnet_1a/egs

print("耗时：",time.time() - ss)

## 单线程 GPU加速， 占用5205MiB的显存

new_06_train_x-vector.sh: Getting neural network training egs
sid/nnet3/xvector/get_egs.sh --cmd run.pl --mem 10G --nj 8 --stage 0 --frames-per-iter 1000000000 --frames-per-iter-diagnostic 100000 --min-frames-per-chunk 100 --max-frames-per-chunk 200 --num-diagnostic-archives 3 --num-repeats 35 data/train_combined_no_sil exp/xvector_nnet_1a/egs
feat-to-dim scp:data/train_combined_no_sil/feats.scp - 
sid/nnet3/xvector/get_egs.sh: Preparing train and validation lists
sid/nnet3/xvector/get_egs.sh: Producing 9 archives for training
1
sid/nnet3/xvector/get_egs.sh: Allocating training examples
sid/nnet3/xvector/get_egs.sh: Allocating training subset examples
sid/nnet3/xvector/get_egs.sh: Allocating validation examples
2
sid/nnet3/xvector/get_egs.sh: Generating training examples on disk
sid/nnet3/xvector/get_egs.sh: Generating training subset examples on disk
sid/nnet3/xvector/get_egs.sh: Generating validation examples on disk
3
sid/nnet3/xvector/get_egs.sh: Shuffling order of archives on disk

2020-11-04 00:12:46,322 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 20/719   Jobs: 1   Epoch: 2.22/80.0 (2.8% complete)   lr: 0.000938   shrink: 0.99062
2020-11-04 00:12:53,632 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 21/719   Jobs: 1   Epoch: 2.33/80.0 (2.9% complete)   lr: 0.000935   shrink: 0.99065
2020-11-04 00:13:00,216 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 22/719   Jobs: 1   Epoch: 2.44/80.0 (3.1% complete)   lr: 0.000932   shrink: 0.99068
2020-11-04 00:13:08,676 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 23/719   Jobs: 1   Epoch: 2.56/80.0 (3.2% complete)   lr: 0.000929   shrink: 0.99071
2020-11-04 00:13:16,200 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 24/719   Jobs: 1   Epoch: 2.67/80.0 (3.3% complete)   lr: 0.000926   shrink: 0.99074
2020-11-04 00:13:26,654 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 25/719   Jobs: 1   Epoch: 2.78/80.0 (3.5% complete)   lr: 0.000923   shrink: 0.99077
2020

2020-11-04 00:19:37,248 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 70/719   Jobs: 1   Epoch: 7.78/80.0 (9.7% complete)   lr: 0.000799   shrink: 0.99201
2020-11-04 00:19:46,191 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 71/719   Jobs: 1   Epoch: 7.89/80.0 (9.9% complete)   lr: 0.000797   shrink: 0.99203
2020-11-04 00:19:56,658 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 72/719   Jobs: 1   Epoch: 8.00/80.0 (10.0% complete)   lr: 0.000794   shrink: 0.99206
2020-11-04 00:20:03,452 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 73/719   Jobs: 1   Epoch: 8.11/80.0 (10.1% complete)   lr: 0.000792   shrink: 0.99208
2020-11-04 00:20:11,263 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 74/719   Jobs: 1   Epoch: 8.22/80.0 (10.3% complete)   lr: 0.000789   shrink: 0.99211
2020-11-04 00:20:17,813 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 75/719   Jobs: 1   Epoch: 8.33/80.0 (10.4% complete)   lr: 0.000787   shrink: 0.99213


2020-11-04 00:26:20,748 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 119/719   Jobs: 1   Epoch: 13.22/80.0 (16.5% complete)   lr: 0.000683   shrink: 0.99317
2020-11-04 00:26:27,306 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 120/719   Jobs: 1   Epoch: 13.33/80.0 (16.7% complete)   lr: 0.000681   shrink: 0.99319
2020-11-04 00:26:34,615 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 121/719   Jobs: 1   Epoch: 13.44/80.0 (16.8% complete)   lr: 0.000679   shrink: 0.99321
2020-11-04 00:26:43,144 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 122/719   Jobs: 1   Epoch: 13.56/80.0 (16.9% complete)   lr: 0.000677   shrink: 0.99323
2020-11-04 00:26:50,673 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 123/719   Jobs: 1   Epoch: 13.67/80.0 (17.1% complete)   lr: 0.000675   shrink: 0.99325
2020-11-04 00:27:01,126 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 124/719   Jobs: 1   Epoch: 13.78/80.0 (17.2% complete)   lr: 0.000673   sh

2020-11-04 00:33:01,505 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 168/719   Jobs: 1   Epoch: 18.67/80.0 (23.3% complete)   lr: 0.000584   shrink: 0.99416
2020-11-04 00:33:11,922 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 169/719   Jobs: 1   Epoch: 18.78/80.0 (23.5% complete)   lr: 0.000582   shrink: 0.99418
2020-11-04 00:33:20,794 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 170/719   Jobs: 1   Epoch: 18.89/80.0 (23.6% complete)   lr: 0.000581   shrink: 0.99419
2020-11-04 00:33:31,259 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 171/719   Jobs: 1   Epoch: 19.00/80.0 (23.8% complete)   lr: 0.000579   shrink: 0.99421
2020-11-04 00:33:38,072 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 172/719   Jobs: 1   Epoch: 19.11/80.0 (23.9% complete)   lr: 0.000577   shrink: 0.99423
2020-11-04 00:33:45,920 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 173/719   Jobs: 1   Epoch: 19.22/80.0 (24.0% complete)   lr: 0.000575   sh

...

2020-11-04 01:20:10,973 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 511/719   Jobs: 1   Epoch: 56.78/80.0 (71.0% complete)   lr: 0.000195   shrink: 0.99805
2020-11-04 01:20:19,871 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 512/719   Jobs: 1   Epoch: 56.89/80.0 (71.1% complete)   lr: 0.000194   shrink: 0.99806
2020-11-04 01:20:30,337 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 513/719   Jobs: 1   Epoch: 57.00/80.0 (71.2% complete)   lr: 0.000194   shrink: 0.99806
2020-11-04 01:20:37,151 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 514/719   Jobs: 1   Epoch: 57.11/80.0 (71.4% complete)   lr: 0.000193   shrink: 0.99807
2020-11-04 01:20:45,013 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 515/719   Jobs: 1   Epoch: 57.22/80.0 (71.5% complete)   lr: 0.000193   shrink: 0.99807
2020-11-04 01:20:51,545 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 516/719   Jobs: 1   Epoch: 57.33/80.0 (71.7% complete)   lr: 0.000192   sh

2020-11-04 01:26:55,127 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 560/719   Jobs: 1   Epoch: 62.22/80.0 (77.8% complete)   lr: 0.000167   shrink: 0.99833
2020-11-04 01:27:02,422 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 561/719   Jobs: 1   Epoch: 62.33/80.0 (77.9% complete)   lr: 0.000166   shrink: 0.99834
2020-11-04 01:27:09,018 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 562/719   Jobs: 1   Epoch: 62.44/80.0 (78.1% complete)   lr: 0.000166   shrink: 0.99834
2020-11-04 01:27:17,517 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 563/719   Jobs: 1   Epoch: 62.56/80.0 (78.2% complete)   lr: 0.000165   shrink: 0.99835
2020-11-04 01:27:25,121 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 564/719   Jobs: 1   Epoch: 62.67/80.0 (78.3% complete)   lr: 0.000165   shrink: 0.99835
2020-11-04 01:27:35,607 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 565/719   Jobs: 1   Epoch: 62.78/80.0 (78.5% complete)   lr: 0.000164   sh

2020-11-04 01:33:36,122 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 609/719   Jobs: 1   Epoch: 67.67/80.0 (84.6% complete)   lr: 0.000143   shrink: 0.99857
2020-11-04 01:33:46,611 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 610/719   Jobs: 1   Epoch: 67.78/80.0 (84.7% complete)   lr: 0.000142   shrink: 0.99858
2020-11-04 01:33:55,517 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 611/719   Jobs: 1   Epoch: 67.89/80.0 (84.9% complete)   lr: 0.000142   shrink: 0.99858
2020-11-04 01:34:06,022 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 612/719   Jobs: 1   Epoch: 68.00/80.0 (85.0% complete)   lr: 0.000141   shrink: 0.99859
2020-11-04 01:34:12,801 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 613/719   Jobs: 1   Epoch: 68.11/80.0 (85.1% complete)   lr: 0.000141   shrink: 0.99859
2020-11-04 01:34:20,673 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 614/719   Jobs: 1   Epoch: 68.22/80.0 (85.3% complete)   lr: 0.000140   sh

2020-11-04 01:40:22,748 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 658/719   Jobs: 1   Epoch: 73.11/80.0 (91.4% complete)   lr: 0.000122   shrink: 0.99878
2020-11-04 01:40:30,528 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 659/719   Jobs: 1   Epoch: 73.22/80.0 (91.5% complete)   lr: 0.000122   shrink: 0.99878
2020-11-04 01:40:37,078 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 660/719   Jobs: 1   Epoch: 73.33/80.0 (91.7% complete)   lr: 0.000121   shrink: 0.99879
2020-11-04 01:40:44,417 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 661/719   Jobs: 1   Epoch: 73.44/80.0 (91.8% complete)   lr: 0.000121   shrink: 0.99879
2020-11-04 01:40:52,973 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 662/719   Jobs: 1   Epoch: 73.56/80.0 (91.9% complete)   lr: 0.000120   shrink: 0.99880
2020-11-04 01:41:00,492 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 663/719   Jobs: 1   Epoch: 73.67/80.0 (92.1% complete)   lr: 0.000120   sh

2020-11-04 01:47:03,006 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 707/719   Jobs: 1   Epoch: 78.56/80.0 (98.2% complete)   lr: 0.000104   shrink: 0.99896
2020-11-04 01:47:10,620 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 708/719   Jobs: 1   Epoch: 78.67/80.0 (98.3% complete)   lr: 0.000104   shrink: 0.99896
2020-11-04 01:47:21,058 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 709/719   Jobs: 1   Epoch: 78.78/80.0 (98.5% complete)   lr: 0.000104   shrink: 0.99896
2020-11-04 01:47:29,940 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 710/719   Jobs: 1   Epoch: 78.89/80.0 (98.6% complete)   lr: 0.000103   shrink: 0.99897
2020-11-04 01:47:40,372 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 711/719   Jobs: 1   Epoch: 79.00/80.0 (98.8% complete)   lr: 0.000103   shrink: 0.99897
2020-11-04 01:47:47,150 [steps/nnet3/train_raw_dnn.py:391 - train - INFO ] Iter: 712/719   Jobs: 1   Epoch: 79.11/80.0 (98.9% complete)   lr: 0.000103   sh

**第七步：** 使用x-vector模型,提取embedding

    1.（train_combined）利用训练好的模型，提取训练集的x-vector 输出位置：exp/xvectors_train_combined
    2. (train_combined) 对数据进行LDA降维  输出位置：exp/xvectors_train_combined/transform.mat
    3. (train_combined) 对数据进行PLDA建模 输出位置：exp/xvectors_train_combined/plda
    

In [18]:
ss= time.time()

!chmod +x new_07_train.sh
!bash new_07_train.sh

print("耗时：",time.time() - ss)

sid/nnet3/xvector/extract_xvectors.sh --cmd run.pl --mem 10G --mem 12G --nj 20 exp/xvector_nnet_1a data/train_combined exp/xvectors_train_combined
sid/nnet3/xvector/extract_xvectors.sh: using exp/xvector_nnet_1a/extract.config to extract xvectors
sid/nnet3/xvector/extract_xvectors.sh: extracting xvectors for data/train_combined
sid/nnet3/xvector/extract_xvectors.sh: extracting xvectors from nnet
sid/nnet3/xvector/extract_xvectors.sh: combining xvectors across jobs
sid/nnet3/xvector/extract_xvectors.sh: computing mean of xvectors for each speaker
耗时： 4647.765430927277


**第八步：** 从测试集中划分训练集和测试集
    
    1.注意线程数量。
    2.划分注册语音和测试语音
    3.分别提取他们的x-vector
    

In [21]:
ss= time.time()

!chmod +x new_08_create_task.sh
!bash new_08_create_task.sh

print("耗时：",time.time() - ss)

utils/fix_data_dir.sh: file data/test/enroll/utt2spk is not in sorted order or not unique, sorting it
fix_data_dir.sh: kept all 60 utterances.
fix_data_dir.sh: old files are kept in data/test/enroll/.backup
utils/fix_data_dir.sh: file data/test/eval/utt2spk is not in sorted order or not unique, sorting it
fix_data_dir.sh: kept all 7116 utterances.
fix_data_dir.sh: old files are kept in data/test/eval/.backup
sid/nnet3/xvector/extract_xvectors.sh --cmd run.pl --mem 10G --mem 10G --nj 20 exp/xvector_nnet_1a data/test exp/xvector_nnet_1a/xvectors_test
sid/nnet3/xvector/extract_xvectors.sh: using exp/xvector_nnet_1a/extract.config to extract xvectors
sid/nnet3/xvector/extract_xvectors.sh: extracting xvectors for data/test
sid/nnet3/xvector/extract_xvectors.sh: extracting xvectors from nnet
sid/nnet3/xvector/extract_xvectors.sh: combining xvectors across jobs
sid/nnet3/xvector/extract_xvectors.sh: computing mean of xvectors for each speaker
耗时： 217.85754323005676


**第九步：** 计算结果
    
    1.提取测试集的表征，因为测试集的数量比较少，使用40进程容易出错。
    2.利用训练好的plda，以及测试集的x-vector生成每一对的得分。
    3.计算EER，minDCF


In [22]:
!chmod +x new_09_result.sh
!bash new_09_result.sh

EER: 0.6745%
minDCF(p-target=0.01): 0.1043
minDCF(p-target=0.001): 0.1816
